In [2]:
!pip install --upgrade pip setuptools wheel
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.4 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.44.0
    Uninstalling wheel-0.44.0:
      Successfully uninstalled wheel-0.44.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)


In [4]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!mkdir -p data
!kaggle datasets download -d grassknoted/asl-alphabet -p data --unzip

Dataset URL: https://www.kaggle.com/datasets/grassknoted/asl-alphabet
License(s): GPL-2.0
100%|█████████████████████████████████████▉| 1.02G/1.03G [00:00<00:00, 3.23GB/s]
100%|██████████████████████████████████████| 1.03G/1.03G [00:00<00:00, 3.30GB/s]


In [9]:
# Inspect the contents of your `data/` folder
import os

for root, dirs, files in os.walk("data"):
    print(f"\nDirectory: {root}")
    if dirs:
        print(" Subfolders:", dirs)
    if files:
        print(" Files (first 10):", files[:10])



Directory: data
 Subfolders: ['asl_alphabet_test', 'asl_alphabet_train']

Directory: data/asl_alphabet_test
 Subfolders: ['asl_alphabet_test']

Directory: data/asl_alphabet_test/asl_alphabet_test
 Files (first 10): ['F_test.jpg', 'G_test.jpg', 'L_test.jpg', 'M_test.jpg', 'R_test.jpg', 'S_test.jpg', 'X_test.jpg', 'Y_test.jpg', 'U_test.jpg', 'T_test.jpg']

Directory: data/asl_alphabet_train
 Subfolders: ['asl_alphabet_train']

Directory: data/asl_alphabet_train/asl_alphabet_train
 Subfolders: ['R', 'U', 'I', 'N', 'G', 'Z', 'T', 'S', 'A', 'F', 'O', 'H', 'del', 'nothing', 'space', 'M', 'J', 'C', 'D', 'V', 'Q', 'X', 'E', 'B', 'K', 'L', 'Y', 'P', 'W']

Directory: data/asl_alphabet_train/asl_alphabet_train/R
 Files (first 10): ['R2837.jpg', 'R2189.jpg', 'R1480.jpg', 'R1494.jpg', 'R2823.jpg', 'R228.jpg', 'R200.jpg', 'R566.jpg', 'R572.jpg', 'R214.jpg']

Directory: data/asl_alphabet_train/asl_alphabet_train/U
 Files (first 10): ['U553.jpg', 'U1601.jpg', 'U2308.jpg', 'U1167.jpg', 'U235.jpg', 'U2

In [15]:
# ── Cell 3: Extract A–Z plus ‘del’ & ‘space’ landmarks ─────────────────────────

import os, cv2, numpy as np
import mediapipe as mp
from tqdm import tqdm

DATA_DIR = "data/asl_alphabet_train/asl_alphabet_train"
if not os.path.isdir(DATA_DIR):
    raise RuntimeError(f"Train directory not found: {DATA_DIR}")

# Only include A–Z, or exactly 'del' or 'space'
valid_labels = set(list("ABCDEFGHIJKLMNOPQRSTUVWXYZ") + ["del", "space"])

all_imgs = []
for label in sorted(os.listdir(DATA_DIR)):
    if label not in valid_labels:
        continue
    folder = os.path.join(DATA_DIR, label)
    for fn in os.listdir(folder):
        if fn.lower().endswith(".jpg"):
            all_imgs.append((label, os.path.join(folder, fn)))

print(f"Found {len(all_imgs)} images across {len(valid_labels)} classes.")

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

X, y = [], []
for label, img_path in tqdm(all_imgs, desc="Extracting landmarks"):
    img = cv2.imread(img_path)
    if img is None: continue
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    res = hands.process(img_rgb)
    if not res.multi_hand_landmarks: continue

    lm = res.multi_hand_landmarks[0]
    coords = []
    for p in lm.landmark:
        coords += [p.x, p.y, p.z]
    X.append(coords)
    y.append(label)

hands.close()

X = np.array(X, dtype=np.float32)
y = np.array(y)
print(f"Extracted landmarks for {len(X)} images (skipped {len(all_imgs)-len(X)}).")


I0000 00:00:1752633901.328267  591779 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3


Found 84000 images across 28 classes.


Extracting landmarks:   0%|          | 0/84000 [00:00<?, ?it/s]W0000 00:00:1752633901.335388  902297 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1752633901.344589  902299 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Extracting landmarks: 100%|██████████| 84000/84000 [27:00<00:00, 51.83it/s] 


Extracted landmarks for 63673 images (skipped 20327).


In [16]:
# ── Cell 4: Encode labels & split data ─────────────────────────────────────────
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

le = LabelEncoder()
y_int = le.fit_transform(y)  # maps 'A'→0, …, 'Z'→25

X_train, X_val, y_train, y_val = train_test_split(
    X, y_int,
    test_size=0.2,
    stratify=y_int,
    random_state=42
)

print("Training set:", X_train.shape, y_train.shape)
print("Validation set:", X_val.shape, y_val.shape)


Training set: (50938, 63) (50938,)
Validation set: (12735, 63) (12735,)


In [ ]:
# ── Cell 5: Build & train the MLP with dynamic output size ────────────────────
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 1) Encode labels and discover number of classes
le = LabelEncoder()
y_int = le.fit_transform(y)        # maps your strings → integers 0…27
num_classes = len(le.classes_)     # should be 28
print("Classes found:", le.classes_)

# 2) Split into train/val
X_train, X_val, y_train, y_val = train_test_split(
    X, y_int,
    test_size=0.2,
    stratify=y_int,
    random_state=42
)
print("Train:", X_train.shape, y_train.shape)
print("Val:  ", X_val.shape,   y_val.shape)

# 3) Build your MLP with the right number of outputs
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(63,)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(num_classes, activation="softmax"),
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

# 4) Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
)


Epoch 1/30


2025-07-15 21:20:22.390637: W tensorflow/core/framework/op_kernel.cc:1857] OP_REQUIRES failed at sparse_xent_op.cc:103 : INVALID_ARGUMENT: Received a label value of 26 which is outside the valid range of [0, 26).  Label values: 7 11 18 8 6 8 21 14 17 12 3 4 19 21 6 4 6 2 16 23 14 10 11 11 12 0 25 26 6 12 24 17


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "/Users/ryan/miniconda3/envs/asl3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/Users/ryan/miniconda3/envs/asl3.9/lib/python3.9/runpy.py", line 87, in _run_code

  File "/Users/ryan/miniconda3/envs/asl3.9/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/Users/ryan/miniconda3/envs/asl3.9/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/Users/ryan/miniconda3/envs/asl3.9/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/Users/ryan/miniconda3/envs/asl3.9/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/Users/ryan/miniconda3/envs/asl3.9/lib/python3.9/asyncio/base_events.py", line 601, in run_fo

In [13]:
model.save("asl_alphabet_mlp.h5")

import pickle
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

print("Saved: asl_alphabet_mlp.h5  and  label_encoder.pkl")


Saved: asl_alphabet_mlp.h5  and  label_encoder.pkl
